# Import dependencies

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

objc[3794]: Class CaptureDelegate is implemented in both /Users/danielhufnagle/anaconda3/envs/HandBot/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x1063c8860) and /Users/danielhufnagle/anaconda3/envs/HandBot/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x2847765a0). One of the two will be used. Which one is undefined.
objc[3794]: Class CVWindow is implemented in both /Users/danielhufnagle/anaconda3/envs/HandBot/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x104a7ca68) and /Users/danielhufnagle/anaconda3/envs/HandBot/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x2847765f0). One of the two will be used. Which one is undefined.
objc[3794]: Class CVView is implemented in both /Users/danielhufnagle/anaconda3/envs/HandBot/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x104a7ca90) and /Users/danielhufnagle/anaconda3/envs/HandBot/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x284776618). O

# Setup for keypoint extraction

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands # hand model
mp_holistic = mp.solutions.holistic

In [3]:
def detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(120, 0, 200), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=4))


In [23]:
vid = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while vid.isOpened():
        _, image = vid.read()
        image, results = detection(image, holistic)
        draw_landmarks(image, results)
        image = cv2.flip(image, 1)
        cv2.imshow('Hands', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
vid.release()
cv2.destroyAllWindows()
# no idea why the hell this is needed, but the capture window won't close otherwise
for i in range(4):
    cv2.waitKey(1)

# Keypoint extraction

In [5]:
def extract_keypoints(results):
    hand_points = np.array([[result.x, result.y, result.z] for result in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return hand_points


In [25]:
vid = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while vid.isOpened():
        _, image = vid.read()
        image, results = detection(image, holistic)
        draw_landmarks(image, results)
        image = cv2.flip(image, 1)
        cv2.imshow('Hands', image)
        print(extract_keypoints(results))
        if cv2.waitKey(0) & 0xFF == ord('q'):
            break
vid.release()
cv2.destroyAllWindows()
# no idea why the hell this is needed, but the capture window won't close otherwise
for i in range(4):
    cv2.waitKey(1)

[ 3.68511826e-01  6.57398820e-01  2.45003122e-07  4.10452813e-01
  6.42237246e-01 -1.69267356e-02  4.47114050e-01  5.98945022e-01
 -2.54390221e-02  4.79119092e-01  5.74330032e-01 -3.38125266e-02
  5.06249666e-01  5.70099413e-01 -4.28358801e-02  4.24842149e-01
  4.87743676e-01 -1.58518590e-02  4.41269904e-01  4.15339887e-01
 -2.74361596e-02  4.49877739e-01  3.70678604e-01 -3.73081751e-02
  4.55901891e-01  3.32419544e-01 -4.54121679e-02  3.98443162e-01
  4.72295970e-01 -1.83724258e-02  4.02959049e-01  3.87080789e-01
 -2.95258258e-02  4.04593617e-01  3.33305359e-01 -4.02721949e-02
  4.04399663e-01  2.89973944e-01 -4.84820083e-02  3.72555375e-01
  4.76070255e-01 -2.29769032e-02  3.71655911e-01  3.91120434e-01
 -3.52893956e-02  3.70001376e-01  3.39419097e-01 -4.61175181e-02
  3.68853509e-01  2.96041846e-01 -5.35905249e-02  3.46950054e-01
  4.97264624e-01 -2.92271767e-02  3.31700623e-01  4.35627997e-01
 -4.29789573e-02  3.20784807e-01  3.94897729e-01 -5.07223979e-02
  3.12566489e-01  3.56514

# Setup for collection

In [6]:
DATA_PATH = os.path.join('MP_Data')

actions = np.array(['thumbs_up', 'thumbs_down', 'open', 'fist', 'peace', 'point', 'middle_finger', 'rock', 'stop', 'okay', 'call_me', 'none'])

In [37]:
os.mkdir('MP_Data')
for action in actions:
    os.mkdir(os.path.join(DATA_PATH, action))

# Collect keypoints

In [28]:
vid = cv2.VideoCapture(0)
base_image_count=0
# collect 250 samples
action = 'call_me' # replace with whatever gesture you want to collect
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while vid.isOpened():
        _, image = vid.read()
        image, results = detection(image, holistic)
        draw_landmarks(image, results)
        image = cv2.flip(image, 1)
        cv2.putText(image, 'Collecting data for ' + action, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (250, 44, 250), 2, cv2.LINE_AA)
        cv2.putText(image, 'Currently extracted ' + str(base_image_count) + ' images', (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (250, 44, 250), 2, cv2.LINE_AA) 
        cv2.imshow('Hands', image)
        keypoints = extract_keypoints(results)
        if cv2.waitKey(1) & 0xFF == ord('f'):
            base_image_count += 1
            print(keypoints)
            npy_path = os.path.join(DATA_PATH, action, str(base_image_count))
            np.save(npy_path, keypoints)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
vid.release()
cv2.destroyAllWindows()
# no idea why the hell this is needed, but the capture window won't close otherwise
for i in range(4):
    cv2.waitKey(1)

[ 2.48875245e-01  9.29898262e-01 -2.58069122e-07  2.62461007e-01
  8.17273915e-01  1.96749135e-03  2.89476603e-01  7.09747374e-01
 -4.59353672e-03  3.01088989e-01  6.29175484e-01 -1.25172408e-02
  2.96000004e-01  5.66591263e-01 -1.74639169e-02  3.35919172e-01
  7.20532358e-01 -1.47252148e-02  3.94451052e-01  7.24027097e-01
 -2.35677361e-02  3.76566708e-01  7.53226161e-01 -2.49764975e-02
  3.54739547e-01  7.61078894e-01 -2.50728875e-02  3.46428812e-01
  7.87473321e-01 -2.18200646e-02  3.99429321e-01  7.88798809e-01
 -2.46553589e-02  3.81745338e-01  8.11686397e-01 -2.03276277e-02
  3.61118793e-01  8.17483604e-01 -1.92630328e-02  3.53313923e-01
  8.56740713e-01 -2.80770417e-02  3.98695469e-01  8.51495743e-01
 -3.02155185e-02  3.84205341e-01  8.72214794e-01 -2.07463894e-02
  3.64563853e-01  8.79911959e-01 -1.56967863e-02  3.57889146e-01
  9.22610700e-01 -3.37919965e-02  4.08835888e-01  9.05214250e-01
 -3.50156091e-02  4.36613351e-01  9.06061828e-01 -2.91921385e-02
  4.59133208e-01  9.02860

# Preprocess data

In [7]:
label_map = {label: num for num, label in enumerate(actions)}

In [8]:
label_map

{'thumbs_up': 0,
 'thumbs_down': 1,
 'open': 2,
 'fist': 3,
 'peace': 4,
 'point': 5,
 'middle_finger': 6,
 'rock': 7,
 'stop': 8,
 'okay': 9,
 'call_me': 10,
 'none': 11}

In [12]:
sequences = []
labels = []
for action in actions:
    for i in range(1, 251):
        sequence = np.load(f'MP_DATA/{action}/{i}.npy')
        labels.append(label_map[action])
        sequences.append(sequence)

In [13]:
print(sequences[:2])
print(labels[:2])

[array([ 1.45001888e-01,  9.02276039e-01, -1.96768326e-07,  1.62515551e-01,
        8.16104770e-01, -6.13933103e-03,  1.99035540e-01,  7.22026765e-01,
       -1.78617612e-02,  2.16518581e-01,  6.51105165e-01, -2.88651008e-02,
        2.16858491e-01,  5.87401867e-01, -3.70842814e-02,  2.10277557e-01,
        7.39072680e-01, -3.85656841e-02,  2.82525867e-01,  7.44795561e-01,
       -5.92343435e-02,  2.66439259e-01,  7.64335275e-01, -6.56759217e-02,
        2.41786644e-01,  7.65295506e-01, -6.70372322e-02,  2.08719641e-01,
        7.99515486e-01, -4.48853709e-02,  2.84795314e-01,  8.08238983e-01,
       -5.81781454e-02,  2.64265627e-01,  8.18955064e-01, -5.26760146e-02,
        2.37772793e-01,  8.15762043e-01, -4.94293347e-02,  2.09016263e-01,
        8.62434447e-01, -5.07631265e-02,  2.76889443e-01,  8.63892972e-01,
       -5.82740679e-02,  2.56354660e-01,  8.70969355e-01, -4.28201593e-02,
        2.32798830e-01,  8.69146407e-01, -3.37088183e-02,  2.09910452e-01,
        9.21491623e-01, 

In [14]:
print(np.array(sequences).shape)
print(np.array(labels).shape)

(3000, 63)
(3000,)


In [15]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [51]:
X_train.shape

(2850, 63)

In [52]:
len(y_train)

2850

In [53]:
y_train[:9]

array([5, 6, 2, 4, 4, 1, 8, 6, 5])

# Build and train model

In [71]:
model = Sequential()
model.add(Dense(64))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(12))

In [72]:
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [82]:
model.fit(X_train, y_train, epochs=50)

Epoch 1/50
90/90 [==============================] - 0s 1ms/step - loss: 0.2032 - accuracy: 0.9179
Epoch 2/50
90/90 [==============================] - 0s 746us/step - loss: 0.2367 - accuracy: 0.9130
Epoch 3/50
90/90 [==============================] - 0s 738us/step - loss: 0.1732 - accuracy: 0.9372
Epoch 4/50
90/90 [==============================] - 0s 740us/step - loss: 0.1652 - accuracy: 0.9400
Epoch 5/50
90/90 [==============================] - 0s 1ms/step - loss: 0.1480 - accuracy: 0.9446
Epoch 6/50
90/90 [==============================] - 0s 704us/step - loss: 0.1355 - accuracy: 0.9530
Epoch 7/50
90/90 [==============================] - 0s 763us/step - loss: 0.1283 - accuracy: 0.9554
Epoch 8/50
90/90 [==============================] - 0s 759us/step - loss: 0.1302 - accuracy: 0.9526
Epoch 9/50
90/90 [==============================] - 0s 776us/step - loss: 0.1096 - accuracy: 0.9625
Epoch 10/50
90/90 [==============================] - 0s 766us/step - loss: 0.1413 - accuracy: 0.9495
Epo

In [83]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)

5/5 - 0s - loss: 0.0073 - accuracy: 1.0000 - 13ms/epoch - 3ms/step


In [84]:
print(test_acc)

1.0


In [85]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = probability_model.predict(X_test)

5/5 [==============================] - 0s 536us/step


In [86]:
np.argmax(predictions[10])

10

In [87]:
y_test[10]

10

In [88]:
model.save('model.h5')

In [10]:
model = tf.keras.models.load_model('model.h5')

In [11]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

# Testing

In [12]:
vid = cv2.VideoCapture(0)
base_image_count=0
# collect 250 samples
action = 'call_me' # replace with whatever gesture you want to collect
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while vid.isOpened():
        _, image = vid.read()
        image, results = detection(image, holistic)
        draw_landmarks(image, results)
        keypoints = extract_keypoints(results)
        image = cv2.flip(image, 1)
        cv2.imshow('Hands', image)
        if cv2.waitKey(1) & 0xFF == ord('f'):
            test_keypoints = keypoints
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
vid.release()
cv2.destroyAllWindows()
# no idea why the hell this is needed, but the capture window won't close otherwise
for i in range(4):
    cv2.waitKey(1)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [13]:
test_keypoints
# this is a thumbs up

array([ 2.91026294e-01,  6.60396814e-01, -3.12734244e-07,  3.06618750e-01,
        5.52211940e-01, -3.01780854e-03,  3.34193468e-01,  4.60472167e-01,
       -1.39917713e-02,  3.53632569e-01,  3.84042323e-01, -2.48551946e-02,
        3.56797308e-01,  3.15484643e-01, -3.31546962e-02,  3.94725055e-01,
        4.81592119e-01, -2.37236340e-02,  4.45243955e-01,  4.95313913e-01,
       -2.99855899e-02,  4.24401581e-01,  5.16404212e-01, -2.89552435e-02,
        4.00001198e-01,  5.20371377e-01, -2.79272534e-02,  4.06281054e-01,
        5.56153297e-01, -2.95789912e-02,  4.48981285e-01,  5.62512040e-01,
       -2.74310093e-02,  4.28082705e-01,  5.75529754e-01, -2.14486104e-02,
        4.05134171e-01,  5.78071654e-01, -2.07128506e-02,  4.10521775e-01,
        6.30644679e-01, -3.41116749e-02,  4.44076180e-01,  6.26110196e-01,
       -3.00442595e-02,  4.24777597e-01,  6.38049960e-01, -1.80512946e-02,
        4.03207690e-01,  6.43007994e-01, -1.43365450e-02,  4.09904152e-01,
        6.99985385e-01, -

In [16]:
test_input = np.array([test_keypoints])
test_input

array([[ 2.91026294e-01,  6.60396814e-01, -3.12734244e-07,
         3.06618750e-01,  5.52211940e-01, -3.01780854e-03,
         3.34193468e-01,  4.60472167e-01, -1.39917713e-02,
         3.53632569e-01,  3.84042323e-01, -2.48551946e-02,
         3.56797308e-01,  3.15484643e-01, -3.31546962e-02,
         3.94725055e-01,  4.81592119e-01, -2.37236340e-02,
         4.45243955e-01,  4.95313913e-01, -2.99855899e-02,
         4.24401581e-01,  5.16404212e-01, -2.89552435e-02,
         4.00001198e-01,  5.20371377e-01, -2.79272534e-02,
         4.06281054e-01,  5.56153297e-01, -2.95789912e-02,
         4.48981285e-01,  5.62512040e-01, -2.74310093e-02,
         4.28082705e-01,  5.75529754e-01, -2.14486104e-02,
         4.05134171e-01,  5.78071654e-01, -2.07128506e-02,
         4.10521775e-01,  6.30644679e-01, -3.41116749e-02,
         4.44076180e-01,  6.26110196e-01, -3.00442595e-02,
         4.24777597e-01,  6.38049960e-01, -1.80512946e-02,
         4.03207690e-01,  6.43007994e-01, -1.43365450e-0

In [20]:
prediction = probability_model.predict(test_input)
arg = np.argmax(prediction)
print(actions[arg])

1/1 [==============================] - 0s 12ms/step
thumbs_up


# Final test

In [22]:
vid = cv2.VideoCapture(0)
base_image_count=0
# collect 250 samples
action = 'call_me' # replace with whatever gesture you want to collect
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while vid.isOpened():
        _, image = vid.read()
        image, results = detection(image, holistic)
        draw_landmarks(image, results)
        keypoints = extract_keypoints(results)
        keypoints = np.array([keypoints])
        prediction = probability_model.predict(keypoints)
        image = cv2.flip(image, 1)
        cv2.putText(image, actions[np.argmax(prediction)], (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (250, 44, 250), 2, cv2.LINE_AA)
        cv2.imshow('Hands', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
vid.release()
cv2.destroyAllWindows()
# no idea why the hell this is needed, but the capture window won't close otherwise
for i in range(4):
    cv2.waitKey(1)

1/1 [==============================] - 0s 9ms/step
